In [1]:
import numpy as np
from netCDF4 import Dataset
from os import listdir
from os.path import isfile, join
import pandas as pd
import pickle
import xarray as xr
import datetime

In [3]:
yearlist = np.arange(2010,2022)
monthlist = ["10","11","12","01","02","03","04"]

awi_dir = '/Users/carmennab/cpom_server/home/cjn/alpha_retracker/data/monthly_SIT_products/AWI_CS2/'

awi_sit = {}

for year in yearlist:
    
    awi_sit[year] = {}
    
    for month in monthlist:
        try:
            d = Dataset(f'{awi_dir}{year}{month}.nc')

            sit = np.array(d['sea_ice_thickness'])[0]

            awi_sit[year][month] = sit
        except:pass

In [4]:
awidir = '/Users/carmennab/cpom_server/home/cjn/alpha_retracker/data/monthly_SIT_products/NSIDC/'

onlyfiles = [f for f in listdir(awidir) if isfile(join(awidir, f))]

yearlist = np.arange(2010,2022)

nsidc_sit = {}

for year in yearlist:
    
    nsidc_sit[year] = {}
    
    for month in monthlist:
        
        nsidc_sit[year][month] = {}
        
        if month == '12':
            m = '01'
        else: 
            m = str(int(month)+1).zfill(2)
        
        s = f'{year}{m}'

        try:
            file = [f for f in onlyfiles if s in f][0]
        except:
            print(s)

        d = Dataset(f'{awidir}{file}')

        sit = np.array(d['sea_ice_thickness'])
        
        sit[sit>100] = np.nan
        sit[sit<0] = np.nan
        
        nsidc_sit[year][month] = sit


201001
201002
201003
201004
201005


In [6]:
cpom_dir = '/Users/carmennab/cpom_server/home/cjn/alpha_retracker/data/monthly_SIT_products/CPOM_CS2/'

cpom_sit = {}

for year in yearlist:
    
    cpom_sit[year] = {}
    
    for month in monthlist:
        try:  
            cpom_sit[year][month] = {}

            df = pd.read_csv(f'{cpom_dir}{year}{month}.txt',sep = ' ')

            thick = np.array(df.iloc[:,2])
            sit = np.reshape(thick, (1335, 1335))

            cpom_sit[year][month] = sit
        except: pass

In [7]:
sit_dict = {'nsidc':nsidc_sit,
            'cpom':cpom_sit,
            'awi':awi_sit}

In [8]:
sit_dict['awi']

{2010: {'10': array([[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]], dtype=float32),
  '11': array([[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]], dtype=float32),
  '12': array([[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)},
 2011: {'10': array([[nan, nan, nan, ..., nan, nan, na

In [9]:
filepath = '/Users/carmennab/cpom_server/home/cjn/alpha_retracker/data/monthly_SIT_products/sits.p'
pickle.dump(sit_dict,open(filepath,'wb'))